<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/main/Paul%20Himyak/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gdown

In [2]:
dataset_url = "https://drive.google.com/uc?export=download&id=1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg"
#dataset_url = "https://docs.google.com/spreadsheets/d/1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg/edit?usp=share_link&ouid=113369956649761615778&rtpof=true&sd=true"
dataset_filename = "profpatolog_modified.xlsx"
#!wget -qc dataset_url -O dataset_filename
gdown.download(dataset_url, output=dataset_filename, quiet=True )


'profpatolog_modified.xlsx'

In [3]:
df = pd.read_excel("profpatolog_modified.xlsx")


In [4]:
#Удаляем случайные вбросы
df.drop([843, 851, 1022, 1024], inplace = True)

#Убираем пустые колонки
empty_columns = df.columns[df.isna().all()]
df.drop(empty_columns, inplace = True, axis = 1)

In [5]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 0
FIELD_CONCLUSION_PART_FIT = 1
FIELD_CONCLUSION_NOT_FIT = 2

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"

In [6]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns


In [99]:
df[mkb_columns] = df[mkb_columns].fillna(value='Z00.0')

In [80]:
pd.get_dummies(df[mkb_columns])

,ПсихиатрияНаркология1_МКБ101_F06.6,ПсихиатрияНаркология1_МКБ101_F06.7,ПсихиатрияНаркология1_МКБ101_F07.0,ПсихиатрияНаркология1_МКБ101_F10.1,ПсихиатрияНаркология1_МКБ101_F41.0,ПсихиатрияНаркология1_МКБ101_F70.0,ПсихиатрияНаркология1_МКБ101_F98.5,ПсихиатрияНаркология1_МКБ101_R25.1,ПсихиатрияНаркология1_МКБ101_Z00.0,Офтальмология1_МКБ101_E11.8,...,Психиатрия1_МКБ102_Z00.0,Психиатрия2_МКБ101_Z00.0,Психиатрия2_МКБ102_Z00.0,Рентгенология1_МКБ102_Z00.0,Рентгенология2_МКБ102_Z00.0,Психиатрия3_МКБ101_Z00.0,Психиатрия3_МКБ102_Z00.0,Психиатрия4_МКБ101_Z00.0,Психиатрия4_МКБ102_Z00.0,Оториноларингология3_МКБ101_Z00.0
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1979,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1980,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:

# объединяем значения из нескольких столбцов в один массив, удаляем дубликаты
#df['mkb'] = df.apply(lambda row: list(set([row[mkb_col_name] for mkb_col_name in mkb_columns if not pd.isna(row[mkb_col_name]) ])), axis=1)
df['mkb'] = df.apply(lambda row: list(set([row[mkb_col_name] for mkb_col_name in mkb_columns])), axis=1)


In [92]:
df['mkb'].values

array([list([nan, 'Z00.0']), list([nan, 'Z00.0', 'I11.9']),
       list([nan, 'Z00.0', 'H52.1']), ..., list([nan, 'K02.1', 'Z00.0']),
       list([nan, 'Z00.0', 'I11.9']), list([nan, 'Z00.0'])], dtype=object)

In [101]:
df['mkb']

0              [Z00.0]
1       [Z00.0, I11.9]
2       [Z00.0, H52.1]
3              [Z00.0]
4       [H52.2, Z00.0]
             ...      
1977           [Z00.0]
1978           [Z00.0]
1979    [K02.1, Z00.0]
1980    [Z00.0, I11.9]
1981           [Z00.0]
Name: mkb, Length: 1978, dtype: object

In [102]:
df['mkb'].explode().unique()

array(['Z00.0', 'I11.9', 'H52.1', 'H52.2', 'H53.0', 'K29.3', 'I70',
       'H90.5', 'G90.9', 'H52.0', 'S68.2', 'N60.2', 'E11.9', 'K29.6',
       'M42.1', 'D25', 'I69.4', 'N83.2', 'H90.3', 'K44.9', 'N80.0',
       'G99.0', 'G12.1', 'K81.9', 'H11.0', 'G25.0', 'K26.9', 'K86.1',
       'H91.1', 'H74.1', 'I67.2', 'N28.1', 'I83.9', 'L40.0', 'T90.4',
       'H93.0', 'D25.9', 'I25.8', 'K80.5', 'F06.6', 'K40.9', 'E11.8',
       'E10.9', 'T94.0', 'F10.7', 'D35.2', 'L20.8', 'K29.5', 'H50.9',
       'K80.2', 'I25.2', 'E14.8', 'I67.9', 'Q12.0', 'S51.9', 'K02.1',
       'K00.0', 'F06.7', 'K03.66', 'K02.0', 'H25.0', 'I66.9', 'K29.8',
       'T92.0', 'F98.5', 'H40.1', 'H90.4', 'F41.0', 'R25.1', 'K25.7',
       'K29.7', 'J45.8', 'H47.2', 'J33.0', 'H66.1', 'K76.0', 'K04.5',
       'K80.1', 'H25.1', 'K25.9', 'K05.4', 'K26.7', 'H02.4', 'I49.9',
       'H01.0', 'I49.8', 'H35.0', 'H17.1', 'K43.6', 'J30.1', 'F07.0',
       'G58.9', 'J37.0', 'I25.9', 'H40.9', 'F70.0', 'G90.8', 'K08.2',
       'K82.8', 'K82.9'

In [116]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.array(df['mkb']))

In [117]:
tokenizer.word_index

{'z00.0': 1,
 'i11.9': 2,
 'h52.1': 3,
 'h52.2': 4,
 'h52.0': 5,
 'k29.3': 6,
 'k02.1': 7,
 'k03.66': 8,
 'h91.1': 9,
 'k29.8': 10,
 'g90.8': 11,
 'h90.3': 12,
 'k29.6': 13,
 'h53.0': 14,
 'k40.9': 15,
 'r25.1': 16,
 'h74.1': 17,
 'k04.5': 18,
 'i67.2': 19,
 'k29.7': 20,
 'm42.1': 21,
 'k86.1': 22,
 'k00.0': 23,
 'h90.4': 24,
 'e11.9': 25,
 'h25.0': 26,
 'd25': 27,
 'd25.9': 28,
 'e11.8': 29,
 'i83.9': 30,
 'l40.0': 31,
 'f06.6': 32,
 't94.0': 33,
 'i25.2': 34,
 'k02.0': 35,
 'j45.8': 36,
 'h47.2': 37,
 'j33.0': 38,
 'k29.4': 39,
 'n60.2': 40,
 't90.4': 41,
 'i67.9': 42,
 'h25.1': 43,
 'k26.7': 44,
 'i25.9': 45,
 'h11.0': 46,
 'g25.0': 47,
 'i25.8': 48,
 'h50.9': 49,
 'k80.2': 50,
 'q12.0': 51,
 'f98.5': 52,
 'h66.1': 53,
 'k25.9': 54,
 'k05.4': 55,
 'h02.4': 56,
 'i49.9': 57,
 'j30.1': 58,
 'k05.3': 59,
 'i70': 60,
 'h90.5': 61,
 'g90.9': 62,
 's68.2': 63,
 'i69.4': 64,
 'n83.2': 65,
 'k44.9': 66,
 'n80.0': 67,
 'g99.0': 68,
 'g12.1': 69,
 'k81.9': 70,
 'k26.9': 71,
 'n28.1': 72,
 'h9

In [118]:
xMkbIndexed = tokenizer.texts_to_sequences(df['mkb'])

In [111]:
mkb_max_len = max(xMkbIndexed, key=len)


In [130]:
import numpy as np

if np.all([isinstance(subarray, list) for subarray in xMkbIndexed]):
    print("All elements of xMkbIndexed are arrays.")
else:
    print("Not all elements of xMkbIndexed are arrays.")


All elements of xMkbIndexed are arrays.


In [131]:
import numpy as np

for i, subarray in enumerate(np.array(xMkbIndexed,dtype=list)):
   if not isinstance(subarray, list):
      print(f"Element at index {i} is not an array: {subarray}")


In [ ]:
xMk

In [136]:
xMkbIndexed

[[1],
 [1, 2],
 [1, 3],
 [1],
 [4, 1],
 [4, 1, 14],
 [1, 3],
 [4, 1],
 [1],
 [1],
 [4, 1],
 [1],
 [1, 3],
 [1, 3],
 [6, 1],
 [1],
 [60, 1],
 [1],
 [61, 6, 1, 3],
 [4, 1],
 [62, 1, 3],
 [1],
 [4, 1],
 [1, 3],
 [1],
 [1],
 [5, 1, 2],
 [5, 63, 1, 2],
 [1, 2],
 [1],
 [1],
 [1],
 [1, 2],
 [5, 40, 1],
 [1],
 [5, 1],
 [1, 3],
 [1],
 [1],
 [1, 3],
 [2, 25, 1, 3],
 [13, 1],
 [1, 2],
 [1, 3],
 [1],
 [1],
 [21, 1],
 [1],
 [4, 1],
 [1],
 [1],
 [5, 1],
 [1, 3],
 [40, 1],
 [1, 3],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1, 3],
 [27, 2, 1, 14],
 [1, 3],
 [1],
 [1, 3],
 [1],
 [27, 1, 3],
 [1],
 [1],
 [1, 3],
 [27, 21, 1],
 [1, 3],
 [1, 3],
 [1],
 [1, 2],
 [1, 3],
 [4, 1],
 [1],
 [1, 2],
 [1],
 [4, 1],
 [1, 3],
 [64, 1],
 [5, 1],
 [1],
 [4, 1],
 [4, 1],
 [65, 1, 3],
 [1, 3],
 [1],
 [1],
 [12, 1],
 [4, 40, 1],
 [2, 1, 3],
 [1, 3],
 [1, 3],
 [1],
 [1],
 [1, 3],
 [1, 2],
 [1, 3],
 [1, 3],
 [27, 1],
 [66, 13, 1, 3],
 [1, 3],
 [1],
 [67, 1, 3],
 [4, 1],
 [1],
 [27, 1, 3],
 [68, 1, 3],
 [1],
 [1, 3],
 [1],
 [5, 1],


In [120]:
xMkbIndexedPadded = pad_sequences(xMkbIndexed, maxlen=mkb_max_len, padding='post')

TypeError: ignored

In [34]:
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: x.replace(".", "x"))

In [29]:
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].str.split(",")
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: ",".join(x))

In [62]:
tokenizer = Tokenizer(filters=" ", split = ",", oov_token = "unknown", char_level = False)

In [63]:
tokenizer.fit_on_texts(df[FIELD_HARMFUL_FACTORS])

In [64]:
xTrainHarmfulIndexes = tokenizer.texts_to_sequences(df[FIELD_HARMFUL_FACTORS])

In [65]:
max_len = max(xTrainHarmfulIndexes, key=len)


In [66]:
xTrainHarmfulIndexes

[[8],
 [5],
 [5],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [21, 41, 2],
 [15, 5, 16, 32],
 [22, 33, 31],
 [22, 33, 31],
 [2],
 [2],
 [2],
 [2],
 [2],
 [6, 4, 12],
 [6, 18],
 [5],
 [6, 4, 12],
 [6, 4, 12],
 [6, 4, 12],
 [6, 12, 4],
 [6, 4, 12],
 [6, 4, 12],
 [6, 4, 12],
 [6],
 [6, 4, 12],
 [6],
 [6, 18],
 [6, 4, 12],
 [6],
 [6, 4, 12],
 [6, 4, 12],
 [2],
 [6, 4, 12],
 [2],
 [4, 12, 13],
 [2],
 [4, 12],
 [4, 12],
 [4, 12],
 [4, 12],
 [4, 12],
 [4, 12],
 [32, 49, 27, 43, 33, 31, 15],
 [32, 49, 27, 43, 33, 31, 15],
 [33, 43, 27, 15, 31, 32, 49],
 [2],
 [32, 49, 27, 43, 33, 31, 15],
 [33, 43, 27, 15, 31, 32, 49],
 [33, 43, 27, 15, 31, 32, 49],
 [33, 43, 27, 15, 31, 32, 49],
 [33, 43, 27, 15, 31, 32, 49],
 [32, 49, 27, 43, 31, 15, 33],
 [32, 49, 27, 33, 31, 15, 43],
 [9],
 [3],
 [21, 41, 2],
 [3],
 [59, 81, 48],
 [3],
 [3],
 [3],
 [3],
 [15, 33, 31],
 [32, 15],
 [32, 15],
 [33, 31, 15],
 [33, 31, 15],
 [33, 31, 15],
 [15, 21],
 [15, 32],
 [33, 31, 15],
 [32, 15],
 [

In [17]:
xTrainHarmfulIndexesPadded = pad_sequences(xTrainHarmfulIndexes, maxlen=max_len, padding='post')

In [14]:
xTrainHarmfulIndexesPadded.shape

(1978, 20)